# Initial import

Let's import what we need for this part and also load all necessary datasets

In [0]:
import pandas as pd
import shutil
import re
import nltk
import random

In [2]:
nltk.download("punkt")
nltk.download("perluniprops")

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
import shutil

drive_path = "Clean REALEC dumps/" #@param {type:"string"}
filename = "realec_110319_2315.tar.gz" #@param {type:"string"}


shutil.copy2('/content/gdrive/My Drive/'+drive_path+filename,'.')

'./realec_110319_2315.tar.gz'

In [0]:
import tarfile

tar = tarfile.open(filename)
tar.extractall()

In [0]:
filename = "All_Entries.json"
shutil.copy2('/content/gdrive/My Drive/'+filename,'.')

All_Entries = pd.read_json(filename).reset_index(drop=True)
All_Entries.begin = All_Entries.begin.astype(int)
All_Entries.end = All_Entries.end.astype(int)
All_Entries.delete = All_Entries.delete.astype(bool)
All_Entries.substr_words = All_Entries.substr_words.astype(int)

In [7]:
%%time

Texts = list(set([x[:-4]+'.txt' for x in list(All_Entries["path"])]))
Text_Dict = {path: open(path, 'r', encoding='utf-8-sig').read() for path in Texts}

CPU times: user 197 ms, sys: 80.6 ms, total: 278 ms
Wall time: 284 ms


In [8]:
print(len(Text_Dict))

5554


In [0]:
filename = "Summary.json"
shutil.copy2('/content/gdrive/My Drive/'+filename,'.')

Summary = pd.read_json(filename)
Summary['Percentage'] = Summary['Percentage'].astype(float)
Summary['Augmented ratio'] = Summary['Augmented ratio'].astype(float)

In [0]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

Selected_Tags = ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
Selected_Tags = [translate_dict[error] for error in Selected_Tags]

filename = "Files_Selected_Errors.json"
shutil.copy2('/content/gdrive/My Drive/'+filename,'.')

Files_Errors = pd.read_json(filename).reset_index(drop=True)

for error in Selected_Tags:
  Files_Errors[error] = Files_Errors[error].astype(int)

# Reserve for final validation

For each of selected types of errors we will reserve some 15 randomly selected texts which contain this specific error. These files are to be used in final validation and thus excerpts of them will not appear in training nor in validation entry subsets.

In [0]:
RESERVE = {Tag: list(Files_Errors['path'].loc[Files_Errors[Tag] > 0].sample(15)) for Tag in Selected_Tags}

In [0]:
for Tag in RESERVE:
  print(Tag+": "+str(RESERVE[Tag]))

Spelling: ['./data/exam/exam2014/EPa_38_1.txt', './data/exam/exam2017_4/DPe_83_1.txt', './data/Exam_practice/OV_2_year/essays_it/st_100_7.txt', './data/exam/exam2014/2012-2014_3/esl_00700.txt', './data/exam/exam2017/EGe_177_2.txt', './data/exam/undefined/EEm_151_1.txt', './data/exam/exam2017/DOv_32_1.txt', './data/2012-2014/esl_00016.txt', './data/old IELTS/IELTS2016/MTsy_40_2.txt', './data/exam/exam2017_2/ABl_18_2.txt', './data/old IELTS/IELTS2016/EEm_200_1.txt', './data/exam/exam2017_6/OBy_5_2.txt', './data/old IELTS/IELTS2015/ESha_49_2.txt', './data/2012-2014/esl_00749.txt', './data/exam/exam2016/JSl_87_1.txt']
lex_item_choice: ['./data/2012-2014/esl_00382.txt', './data/exam/best_works/2_2.txt', './data/2012-2014/esl_01177.txt', './data/exam/exam2016/EKu_12_2.txt', './data/exam/exam2017/VSa_35_1.txt', './data/exam/exam2017_6/OBy_69_2.txt', './data/exam/exam2014/AAl_9_2.txt', './data/exam/exam2014/EEm_36_1.txt', './data/Exam_practice/OV_2_year/essays_fr/st_45_7.txt', './data/old IELT

# Collecting erroneous entries

In [0]:
%%time

ENTRIES_DATASETS = {}

for Tag in Selected_Tags:
  if Tag == "delete":
    _entries = All_Entries.loc[(All_Entries['delete'] == True) & (All_Entries['substr_words'] == 1)]
  else:
    _entries = All_Entries.loc[(All_Entries['type'] == Tag) & (All_Entries['substr_words'] == 1)]
  for path in RESERVE[Tag]:
    _entries = _entries.loc[_entries['path'] != path[:-4]+".ann"]
  ENTRIES_DATASETS[Tag] = _entries

CPU times: user 655 ms, sys: 3.73 ms, total: 658 ms
Wall time: 678 ms


In [0]:
ENTRIES_DATASETS["delete"].sample(5)

,path,id,type,begin,end,substring,correction,delete,context,substr_words,unmasked_context
42163,./data/exam/exam2016/OR_10_2.ann,T44,Redundant_comp,1605,1608,all,,True,However; laws to reduce the amount of air trav...,1,However; laws to reduce the amount of air trav...
23135,./data/exam/exam2017/OBy_134_2.ann,T6,Redundant_comp,203,207,only,,True,Someone thinks that countries only need to pro...,1,Someone thinks that countries only need to pro...
25539,./data/exam/exam2017/OBy_62_2.ann,T28,Redundant_comp,655,659,name,,True,Machines analyzing our skin by the our past ex...,1,Machines analyzing our skin by the our past ex...
52799,./data/exam/exam2017_6/OBy_63_2.ann,T23,Redundant_comp,428,434,around,,True,"It is very big problem, that we can not marge ...",1,"It is very big problem, that we can not marge ..."
108062,./data/old IELTS/IELTS2015/EPa_19_2.ann,T7,Redundant_comp,399,402,who,,True,For good understanding for this view I want to...,1,For good understanding for this view I want to...


# Generating non-erroneous entries

We'll create a bunch of non-erroneous entries for each type of error. The number of such entries will be the maximum of `15 * entries_of_this_error_type` and `1 / augmented_ratio_for_this_error_type * entries_of_this_error_type`, we'll just randomly select the lesser necessary amount later. To avoid the possibility of randomly encountering a word which actually is the error of our type we will check that our randomly generated entry does not appear in erroneous contexts. We will also check that our contexts don't match with any of non-one word contexts of the same error type, just for precaution sake.

In [0]:
def random_mask_one(intext):
  tokenssoup = tknzr.tokenize(intext)
  chosen = randint(0, len(tokenssoup)-1)
  for i in range(chosen):
    intext = intext.replace(tokenssoup[i], chr(8), 1)
  intext = intext.replace(tokenssoup[chosen], chr(8), 1)
  for i in range(chosen):
    intext = intext.replace(chr(8), tokenssoup[i], 1)
  spacechars = chr(9)+chr(10)+chr(160)+" "
  if not re.search(r'['+spacechars+']'+chr(8), intext):
    intext = intext.replace(chr(8), ' '+chr(8))
  if not re.search(chr(8)+r'['+spacechars+']', intext):
    intext = intext.replace(chr(8), chr(8)+' ')
  intext = intext.replace(chr(8), "[MASK]")
  sentences = nltk.sent_tokenize(intext)
  for k in range(len(sentences)):
    if '[MASK]' in sentences[k]:
      if k < 1:
        return ' '.join(sentences[0:k+2]), tokenssoup[chosen]
      else:
        return ' '.join(sentences[k-1:k+2]), tokenssoup[chosen]
      break

In [0]:
from math import ceil

In [0]:
%%time

NOERROR_DATASETS = {}

for Tag in Selected_Tags:
  _Path = []
  _Context = []
  _Substring = []
  _err_contexts = set(ENTRIES_DATASETS[Tag]["context"])
  if Tag == "delete":
    _unmasked_contexts = set(All_Entries.loc[(All_Entries['delete'] == True) & (All_Entries['substr_words'] != 1)]["unmasked_context"])
  else:
    _unmasked_contexts = set(All_Entries.loc[(All_Entries['type'] == Tag) & (All_Entries['substr_words'] != 1)]["unmasked_context"])
  
  err_contexts = len(ENTRIES_DATASETS[Tag])
  contexts_needed = max(err_contexts * 15, ceil(err_contexts * (1/Summary["Augmented ratio"][Tag])))
  
  i = 0
  while (i < contexts_needed):
    _path = random.choice(Texts)
    if _path in RESERVE[Tag]:
      continue
    context, substring = random_mask_one(Text_Dict[_path])
    if context in _err_contexts:
      continue
    if context.replace("[MASK]", substring) in _unmasked_contexts:
      continue
    _Path.append(_path)
    _Context.append(context)
    _Substring.append(substring)
    i += 1
  
  NOERROR_DATASETS[Tag] = pd.DataFrame({
      "path": _Path,
      "context": _Context,
      "substring": _Substring
  })

CPU times: user 54min 10s, sys: 2.25 s, total: 54min 12s
Wall time: 54min 29s


We better save it off the bat.

In [0]:
%%time

for jsonname in Selected_Tags:
  with open(jsonname+".json", 'w', encoding="utf-8") as outie:
    outie.write(NOERROR_DATASETS[jsonname].to_json())

CPU times: user 2.97 s, sys: 1.11 s, total: 4.08 s
Wall time: 4.39 s


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Create & upload a file.

for jsonname in Selected_Tags:
  uploaded = drive.CreateFile({'title': jsonname+".json"})
  uploaded.SetContentFile(jsonname+".json")
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1cFHqKQVBwsICImDLAExNiD1cIGW1Y1V2
Uploaded file with ID 1TBcIrjzOvF5_dmxIkRmUuVFNzMuoI65i
Uploaded file with ID 1yCWQSRutdo4eEmJcSgGno8r89CtPKZX1
Uploaded file with ID 1sKi6SOnPncLgGTpSMsBSYrr4Ukua0m5x
Uploaded file with ID 1cX8QHls87wPNgvPgqvV6KTNkE0gxIeBH
Uploaded file with ID 17f9HF5fOfEep8rFjA05pqWqbOl8qaWa9
Uploaded file with ID 1tRimxmFOm5xPz9sddjcTjdrjHGn4oRlL
Uploaded file with ID 1c6lKEcYM-KVSf8HQvaIl3dPIbse35ATJ
Uploaded file with ID 1kM2dUilTXes2P8kGWUSO9Vr5AaQoazx6
Uploaded file with ID 16kEGsBzihQyE-cDPOc92Rl4ItJpEetAy


# Preparing resulting files

Around here we should get rid of unnecessary columns in erroneous datasets. We'll keep file paths for now just to keep track. At this point we also divide the data into training and test sets (80% and 20%, keeping the erroneous to non-erroneous ratio at constant in each subset).

In [0]:
for Tag in Selected_Tags:
  ENTRIES_DATASETS[Tag].drop(["id", "type", "begin", "end", "correction", "delete", "substr_words", "unmasked_context"], axis=1, inplace=True)
  ENTRIES_DATASETS[Tag]["is_error"] = 1
  NOERROR_DATASETS[Tag]["is_error"] = 0

In [0]:
# This also randomizes them, so we can just divide them with slices next

MainDatasets = {Tag: ENTRIES_DATASETS[Tag] for Tag in Selected_Tags}
ToFifteen_Complements = {Tag: NOERROR_DATASETS[Tag].sample(len(ENTRIES_DATASETS[Tag])*15).reset_index(drop=True) for Tag in Selected_Tags}
AugmentedRatio_Complements = {Tag: NOERROR_DATASETS[Tag].sample(ceil(len(ENTRIES_DATASETS[Tag]) * (1/Summary["Augmented ratio"][Tag]))).reset_index(drop=True) for Tag in Selected_Tags}

In [0]:
ToFifteen = {Tag: {"train": pd.concat([MainDatasets[Tag][:round(len(MainDatasets[Tag])*0.8)], ToFifteen_Complements[Tag][:round(len(ToFifteen_Complements[Tag])*0.8)]]).reset_index(drop=True), "test": pd.concat([MainDatasets[Tag][round(len(MainDatasets[Tag])*0.8):], ToFifteen_Complements[Tag][round(len(ToFifteen_Complements[Tag])*0.8):]]).reset_index(drop=True)} for Tag in Selected_Tags}
AugmentedRatio = {Tag: {"train": pd.concat([MainDatasets[Tag][:round(len(MainDatasets[Tag])*0.8)], AugmentedRatio_Complements[Tag][:round(len(AugmentedRatio_Complements[Tag])*0.8)]]).reset_index(drop=True), "test": pd.concat([MainDatasets[Tag][round(len(MainDatasets[Tag])*0.8):], AugmentedRatio_Complements[Tag][round(len(AugmentedRatio_Complements[Tag])*0.8):]]).reset_index(drop=True)} for Tag in Selected_Tags}

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Let's now save all our precious freshly-generated datasets.

In [0]:
%%time

for jsonname in Selected_Tags:
  with open(jsonname+".json", 'w', encoding="utf-8") as outie:
    outie.write(AugmentedRatio[jsonname]["test"].to_json())

CPU times: user 657 ms, sys: 369 ms, total: 1.03 s
Wall time: 1.04 s


In [0]:
import json

category_confusion_files = ['./data/exam/exam2014/EEm_7_1.txt', './data/old IELTS/IELTS2016/EKu_145_1.txt', './data/exam/exam2016/best_works/ZEv_2_2.txt', './data/exam/exam2017_2/ABl_26_2.txt', './data/exam/exam2017/OBy_149_2.txt', './data/exam/exam2017_5_2/EGe_226_2.txt', './data/exam/exam2017_5_2/EGe_146_1.txt', './data/old IELTS/IELTS2016/EKu_61_1.txt', './data/exam/exam2014/AAl_24_2.txt', './data/exam/exam2016/ZEv_52_2.txt', './data/exam/exam2014/AAl_31_2.txt', './data/exam/exam2017_4/DPe_83_1.txt', './data/old IELTS/IELTS2016/EKu_132_1.txt', './data/old IELTS/IELTS2015/EEm_38_1.txt', './data/exam/exam2014/MBi_22_1.txt']
category_confusion_dict = {path: Text_Dict[path] for path in category_confusion_files}
with open("category_confusion_dict.json", "w") as outjson:
  json.dump(category_confusion_dict, outjson)

In [10]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.8MB/s 


In [11]:
# Create & upload a file.

jsonname = "category_confusion_dict"
uploaded = drive.CreateFile({'title': jsonname+".json"})
uploaded.SetContentFile(jsonname+".json")
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1wS1SrvAOpE6TDl2pbWDDs1drYoOrDabH


In [0]:
# Create & upload a file.

uploaded = drive.CreateFile({'title': jsonname+".json"})
  uploaded.SetContentFile(jsonname+".json")
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1a7ojnrobOMIBGiaiNH7IASDpu52XUg-2
Uploaded file with ID 1WJq-oFRAizePrOtYYHp4MJn5ULcOGpff
Uploaded file with ID 1imMW4-z_JONepWYCho1dlRQL6reSBOau
Uploaded file with ID 1_BhAHeVBBgJqxHFSyZm-uPnv3j7n7Plo
Uploaded file with ID 1HBsUINkt-wx5dvrWXjWSvaVo2f9lvK5s
Uploaded file with ID 1_64bYqZTfCi1TRn7QmxSJKh-eaNtW0bd
Uploaded file with ID 1psaSVmvO86QBEbhLNHk2N4iIRdg6Fysr
Uploaded file with ID 11g3GtCdf9NRZXRNi3XohZjBscQ8BZUUo
Uploaded file with ID 1xj6z47TrOrh8IOp-frKCXo7ed5ej6KDy
Uploaded file with ID 1UANXhnjWyxrzil_h7EreKoj24EXLVnfx
